In [1]:
# /Users/teliov/TUD/Thesis/Medvice/Notebooks/data/04_06_new_data/data/split
# So we can use the *thesislib* package
import sys
import os

module_path = os.path.abspath("..")

if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from thesislib.utils import pathutils

In [10]:
basic_train_csv = pathutils.get_data_file('05_27_nlice/basic/data/parsed/train.csv_sparse.csv')
symptom_map_file = pathutils.get_data_file("05_27_nlice/basic/symptom_db.json")

In [4]:
import pandas as pd
import numpy as np

In [5]:
data = pd.read_csv(basic_train_csv, index_col="Index")

In [7]:
labels = data.LABEL
ordered_keys = ['GENDER', 'RACE', 'AGE', 'SYMPTOMS']
data = data[ordered_keys]

In [9]:
from thesislib.utils.ml import models

In [11]:
import json
with open(symptom_map_file) as fp:
    symptom_db = json.load(fp)

num_symptoms = len(symptom_db)

In [12]:
sparsifier = models.ThesisSymptomSparseMaker(num_symptoms=num_symptoms)

In [13]:
sparse_data = sparsifier.fit_transform(data)

In [16]:
import lightgbm as lgb

In [45]:
lgb_model = lgb.LGBMClassifier(
    boosting_type='gbdt',
    num_leaves=100,
    max_depth=- 1,
    learning_rate=0.1,
    n_estimators=1000,
    subsample_for_bin=200000,
    objective=None,
    class_weight=None,
    min_split_gain=0.0,
    min_child_weight=0.001,
    min_child_samples=20,
    subsample=1.0,
    subsample_freq=0,
    colsample_bytree=1.0,
    reg_alpha=0.0,
    reg_lambda=0.0,
    random_state=None,
    n_jobs=- 1,
    silent=True,
    importance_type='split'
)

In [46]:
from sklearn.model_selection import StratifiedShuffleSplit
split_t = StratifiedShuffleSplit(n_splits=1, test_size=0.2)

train_data = None
train_labels = None
test_data = None
test_labels = None
for train_index, test_index in split_t.split(sparse_data, labels):
    train_data = sparse_data[train_index]
    train_labels = labels[train_index]
    test_data = sparse_data[test_index]
    test_labels = labels[test_index]

In [47]:
from thesislib.utils import stringutils
symptoms = [stringutils.slugify(symptom_db[item]) for item in sorted(symptom_db.keys())]

In [48]:
feature_names = ['gender', 'race', 'age'] + symptoms
categorical_data = ['gender', 'race', 'headache', 'limb-weakness', 'abdominal-pain', 'fever']

In [49]:
lgb_model = lgb_model.fit(
    train_data,
    train_labels,
    #feature_name=feature_names,
    #categorical_feature=categorical_data
)

In [50]:
pred =  lgb_model.predict(train_data)

In [51]:
from sklearn.metrics import accuracy_score

In [52]:
accuracy_score(train_labels, pred)

0.19276315789473683